## ERA5 correction 
Adapted from Eric Gagliano's notebook here: https://github.com/gbrencher/AMATH563-InSAR-denoiser/blob/main/data-processing/era5_correction_individual_igrams.ipynb

Note: This code will only work if in the MintPy source code (tropo_pyaps3.py) we comment out lines 657 (ref_y, ref_x = int(atr['REF_Y']), int(atr['REF_X'])) and 666 (data -= data[ref_y, ref_x]). This code subtracts out the value at the reference point. Since we haven't set a reference point in the metadata of these interferograms, it won't work. We can handle this later

In [7]:
import mintpy
from pathlib import Path
from dateutil.parser import parse as parse_date
from pathlib import Path
from typing import List, Union
from osgeo import gdal
from pathlib import Path
from typing import List, Union
import os
import glob
import rasterio
import matplotlib.pyplot as plt
from mintpy.utils import ptime, readfile, writefile, utils as ut

In [8]:
weather_dir = '/home/jovyan/InSAR_denoising_CNN/era5_files'
directories = glob.glob('/home/jovyan/InSAR_denoising_CNN/asc_crop_sanjuans_noise/S1*/')

In [9]:
for i, idir in enumerate(directories):
    os.chdir(idir)         # Change working Directory
    print(f'working on {i}/{len(directories)}')
    !pwd
    data_dir = Path.cwd()
    mintpy_config = data_dir / 'mintpy_config.txt'
    mintpy_config.write_text(
    f"""
    mintpy.load.processor        = hyp3
    ##---------interferogram datasets:
    mintpy.load.unwFile          = *_unw_phase.tif
    mintpy.load.corFile          = *_corr.tif
    ##---------geometry datasets:
    mintpy.load.demFile          = *_dem.tif
    mintpy.load.incAngleFile     = *_lv_theta.tif
    mintpy.load.azAngleFile      = *_lv_phi.tif
    mintpy.load.waterMaskFile    = *_water_mask.tif
    """)
    !smallbaselineApp.py --dir {data_dir} {mintpy_config} --dostep load_data
    geom_path = 'inputs/geometryGeo.h5'

    tif_filename = glob.glob('*_unw_phase.tif')[0] #clipped
    unw_filename = f'{tif_filename[:-4]}.unw'

    data, atr = readfile.read(tif_filename, datasetName='phase')
    #atr['OG_FILE_PATH'] = atr['FILE_PATH']
    atr['FILE_TYPE'] = '.unw'
    writefile.write({'phase':data},unw_filename,metadata=atr)
    #!gdal_translate $tif_filename $unw_filename -of ISCE
    #print(unw_filename)
    #!pwd
    #!gdalinfo $unw_filename
    !tropo_pyaps3.py -f $unw_filename -g $geom_path --weather-dir $weather_dir
    unw_filename_corr = f'{unw_filename[:-4]}_ERA5.unw'
    unw_filename_out = f'{unw_filename[:-4]}_ERA5.tif'
    !gdal_translate -of GTiff -b 2 -a_srs EPSG:32613 $unw_filename_corr $unw_filename_out
    !rm *.rsc
    !rm *.unw
    !rm -r inputs
    !rm -r pic
    !rm smallbaselineApp.cfg
    !rm mintpy_config.txt

working on 0/41
/home/jovyan/InSAR_denoising_CNN/asc_crop_sanjuans_noise/S1BB_20180807T010905_20180819T010906_VVP012_INT40_G_ueF_05F4
MintPy version 1.4.1, date 2022-08-15
--RUN-at-2022-11-01 20:56:20.247510--
Current directory: /home/jovyan/InSAR_denoising_CNN/asc_crop_sanjuans_noise/S1BB_20180807T010905_20180819T010906_VVP012_INT40_G_ueF_05F4
Run routine processing with smallbaselineApp.py on steps: ['load_data']
Remaining steps: ['modify_network', 'reference_point', 'quick_overview', 'correct_unwrap_error', 'invert_network', 'correct_LOD', 'correct_SET', 'correct_troposphere', 'deramp', 'correct_topography', 'residual_RMS', 'reference_date', 'velocity', 'geocode', 'google_earth', 'hdfeos5']
--------------------------------------------------
Project name: mintpy_config
Go to work directory: /home/jovyan/InSAR_denoising_CNN/asc_crop_sanjuans_noise/S1BB_20180807T010905_20180819T010906_VVP012_INT40_G_ueF_05F4
copy default template file /home/jovyan/.local/envs/insar_analysis/lib/python3